In [2]:
library(here)
library(here)
library(readr, warn.conflicts = FALSE)
library(arrow, warn.conflicts = FALSE)
library(magrittr, warn.conflicts = FALSE)
library(stringr, warn.conflicts = FALSE)
library(dplyr, warn.conflicts = FALSE)
library(rlang, warn.conflicts = FALSE)
library(data.table, warn.conflicts = FALSE)
library(lubridate, warn.conflicts = FALSE)
library(tidyr, warn.conflicts = FALSE)
library(truveta.study)
library(dbplyr)
library(ggplot2)
# build connections 
con <- create_connection()
study <- get_study(con, title = "Hypertension Cohort")
population <- get_population(con, study, title = "Study Cohort")
snapshot <- get_latest_snapshot(con, population, finished = TRUE)

here() starts at /home/jovyan/studyrepositories/hypertensioncohorts-is5fyzbqztbebenalfbuo4boem

Loading required package: DBI


Attaching package: ‘dbplyr’


The following objects are masked from ‘package:dplyr’:

    ident, sql




[1] "Deprecation warning: the Vital and SocialHistory tables will be removed from the Truveta Data Model on September 12th, 2023. New snapshots generated after this date will exclude these tables. Existing snapshots which include these tables will be unaffected. Visit Learn for more information."


To sign in to Truveta, use a web browser to open the page https://login.truveta.com/activate?user_code=SBFT-SDFK.

Signed in successfully.



In [4]:
#get cleaned body weight vital data in ambulatory and inpatient from Observation
param1 = list( ClassConcept=1065220) # inpatient
param2 = list( ClassConcept=1065216) #outpatient
concept <- read_parquet("data/Concept.parquet")

In [15]:
# encounter type by vital sign type

sql <- "select v.CodeConceptId, e.ClassConceptId, count(distinct v.Id) as NonMissing
from Vital v join Encounter e on (v.EncounterId = e.Id)
where v.Value is not NULL and v.EffectiveDateTime is not NULL
and e.StatusConceptId not in (2983199,2506590,2983200,2506591,2506595)
group by CodeConceptId, ClassConceptId
order by 1,2;"

ev <- load_sql_table(con,snapshot, query = sql) %>% collect()

#ev
ev_names <- ev %>% merge(.,concept, by.x = "CodeConceptId", by.y = "ConceptId")%>%
select(VitalSignId = CodeConceptId, ClassConceptId, VitalSignName =ConceptName, NonMissing)%>%
merge(.,concept,by.x= 'ClassConceptId',by.y = 'ConceptId')%>%
select(VitalSignId, VitalSignName, EncounterClassId = ClassConceptId, EncounterClassName = ConceptName, NonMissing)

write_csv(ev_names,"data/ev_names_distribution.csv.r")

In [5]:
#get cleaned body height data from Observation
# inpatient
sql <- "select v.*, e.ClassConceptId
from Observation v join Encounter e on (v.EncounterId = e.Id)
where e.ClassConceptId = :ClassConcept and v.CodeConceptId  = 779602
and v.NormalizedValueNumeric is not NULL
and NormalizedValueUOMConceptId not in (0,1067557,1067561)
and v.EffectiveDateTime is not NULL;  "


inp_bh_obs <- load_sql_table(con,snapshot,query = sql, params = param1) %>% collect()
# outpatient
sql <- "select v.*, e.ClassConceptId
from Observation v join Encounter e on (v.EncounterId = e.Id)
where e.ClassConceptId = :ClassConcept and v.CodeConceptId  = 779602
and v.NormalizedValueNumeric is not NULL
and NormalizedValueUOMConceptId not in (0,1067557,1067561)
and v.EffectiveDateTime is not NULL;  "


amb_bh_obs <- load_sql_table(con,snapshot,query = sql, param2) %>% collect()


In [6]:
# analyze Height data

inp_bh_obs %>% group_by(PersonId) %>% summarise(counts = n_distinct(Id))%>% mutate(sd = sd(counts))%>% summary()
amb_bh_obs %>% group_by(PersonId) %>% summarise(counts = n_distinct(Id))%>% mutate(sd = sd(counts))%>% summary()

   PersonId             counts              sd       
 Length:43622       Min.   :   1.00   Min.   :46.21  
 Class :character   1st Qu.:   7.00   1st Qu.:46.21  
 Mode  :character   Median :  12.00   Median :46.21  
                    Mean   :  23.94   Mean   :46.21  
                    3rd Qu.:  27.00   3rd Qu.:46.21  
                    Max.   :2863.00   Max.   :46.21  

   PersonId             counts             sd       
 Length:55349       Min.   :  1.00   Min.   :15.01  
 Class :character   1st Qu.:  2.00   1st Qu.:15.01  
 Mode  :character   Median :  4.00   Median :15.01  
                    Mean   : 10.35   Mean   :15.01  
                    3rd Qu.: 12.00   3rd Qu.:15.01  
                    Max.   :263.00   Max.   :15.01  

In [7]:
inp_bh_obs %>% group_by(NormalizedValueUOMConceptId) %>% summarise(mean = mean(NormalizedValueNumeric),
                                                                   min = min(NormalizedValueNumeric),
                                                                   qr1 = quantile(NormalizedValueNumeric,0.25),
                                                                   median = quantile(NormalizedValueNumeric,0.5),
                                                                   qr3 = quantile(NormalizedValueNumeric, 0.75),
                                                                   max = max(NormalizedValueNumeric),
                                                                   sd = sd(NormalizedValueNumeric))%>%
merge(.,concept,by.x = NormalizedValueUOMConceptId, by.y = ConceptId)%>%
select(ConceptId, ConceptName, mean, min,qr1,median,qr3,max,sd)
# amb_bh_obs 

ERROR: Error in fix.by(by.x, x): object 'NormalizedValueUOMConceptId' not found


In [ ]:
#get cleaned body weight vital data in ambulatory and inpatient from Observation
param1 = list( ClassConcept=1065220) # inpatient
param2 = list( ClassConcept=1065216) #outpatient
sql1 <- "select 
v.*, e.ClassConceptId
from Observation v join Encounter e on (v.EncounterId = e.Id)
where e.ClassConceptId = :ClassConcept and v.CodeConceptId =779749
and v.NormalizedValueNumeric is not NULL
and NormalizedValueUOMConceptId not in (0,1067557,1067561)
and e.StatusConceptId not in (2983199,2506590,2983200,2506591,2506595)
and v.EffectiveDateTime is not NULL;
"
sql2 <- "select 
v.*, e.ClassConceptId
from Vital v join Encounter e on (v.EncounterId = e.Id)
where e.ClassConceptId = :ClassConcept and v.CodeConceptId in (800318,773683)
and v.NormalizedValueNumeric is not NULL
and NormalizedValueUOMConceptId not in (0,1067557,1067561)
and e.StatusConceptId not in (2983199,2506590,2983200,2506591,2506595)
and v.EffectiveDateTime is not NULL;
"
inpatient_dbp <- load_sql_table(con,snapshot,query = sql1,params = param1) %>% collect()
amb__bw_bh_vital <- load_sql_table(con,snapshot, query = sql2,params = param2) %>% collect()
head(inpatient_bw_bh_vital)
head(amb__bw_bh_vital)

In [1]:
#get cleaned body blood pressure data from Observation
# inpatient dbp
sql <- "select v.*, e.ClassConceptId
from Observation v join Encounter e on (v.EncounterId = e.Id)
where e.ClassConceptId = :ClassConcept and v.CodeConceptId  = 779749
and v.NormalizedValueNumeric is not NULL
and NormalizedValueUOMConceptId not in (0,1067557,1067561)
and e.StatusConceptId not in (2983199,2506590,2983200,2506591,2506595)
and v.EffectiveDateTime is not NULL;  "

inp_dbp <- load_sql_table(con,snapshot,query = sql, params = param1) %>% collect()
# outpatient dbp
sql <- "select v.*, e.ClassConceptId
from Observation v join Encounter e on (v.EncounterId = e.Id)
where e.ClassConceptId = :ClassConcept and v.CodeConceptId  = 779749
and v.NormalizedValueNumeric is not NULL
and NormalizedValueUOMConceptId not in (0,1067557,1067561)
and e.StatusConceptId not in (2983199,2506590,2983200,2506591,2506595)
and v.EffectiveDateTime is not NULL;  "


amb_dbp <- load_sql_table(con,snapshot,query = sql, param2) %>% collect()


ERROR: Error in load_sql_table(con, snapshot, query = sql, params = param1) %>% : could not find function "%>%"


In [1]:
#get cleaned body blood pressure data from Observation
# inpatient sbp
sql <- "select v.*, e.ClassConceptId
from Observation v join Encounter e on (v.EncounterId = e.Id)
where e.ClassConceptId = :ClassConcept and v.CodeConceptId  = 779767
and v.NormalizedValueNumeric is not NULL
and NormalizedValueUOMConceptId not in (0,1067557,1067561)
and e.StatusConceptId not in (2983199,2506590,2983200,2506591,2506595)
and v.EffectiveDateTime is not NULL;  "

inp_sbp <- load_sql_table(con,snapshot,query = sql, params = param1) %>% collect()
# outpatient dbp
sql <- "select v.*, e.ClassConceptId
from Observation v join Encounter e on (v.EncounterId = e.Id)
where e.ClassConceptId = :ClassConcept and v.CodeConceptId  = 779767
and v.NormalizedValueNumeric is not NULL
and NormalizedValueUOMConceptId not in (0,1067557,1067561)
and e.StatusConceptId not in (2983199,2506590,2983200,2506591,2506595)
and v.EffectiveDateTime is not NULL;  "


amb_sbp <- load_sql_table(con,snapshot,query = sql, param2) %>% collect()


ERROR: Error in load_sql_table(con, snapshot, query = sql, params = param1) %>% : could not find function "%>%"


In [1]:
#get cleaned body blood pressure data from Observation
# inpatient bp
sql <- "select v.*, e.ClassConceptId
from Observation v join Encounter e on (v.EncounterId = e.Id)
where e.ClassConceptId = :ClassConcept and v.CodeConceptId  = 825889
and v.NormalizedValueNumeric is not NULL
and NormalizedValueUOMConceptId not in (0,1067557,1067561)
and e.StatusConceptId not in (2983199,2506590,2983200,2506591,2506595)
and v.EffectiveDateTime is not NULL;  "

inp_bp <- load_sql_table(con,snapshot,query = sql, params = param1) %>% collect()
# outpatient bp
sql <- "select v.*, e.ClassConceptId
from Observation v join Encounter e on (v.EncounterId = e.Id)
where e.ClassConceptId = :ClassConcept and v.CodeConceptId  = 825889
and v.NormalizedValueNumeric is not NULL
and NormalizedValueUOMConceptId not in (0,1067557,1067561)
and e.StatusConceptId not in (2983199,2506590,2983200,2506591,2506595)
and v.EffectiveDateTime is not NULL;  "


amb_bp <- load_sql_table(con,snapshot,query = sql, param2) %>% collect()


ERROR: Error in load_sql_table(con, snapshot, query = sql, params = param1) %>% : could not find function "%>%"


In [6]:
# analyze blood pressure data

inp_bh_obs %>% group_by(PersonId) %>% summarise(counts = n_distinct(Id))%>% mutate(sd = sd(counts))%>% summary()
amb_bh_obs %>% group_by(PersonId) %>% summarise(counts = n_distinct(Id))%>% mutate(sd = sd(counts))%>% summary()

   PersonId             counts              sd       
 Length:43622       Min.   :   1.00   Min.   :46.21  
 Class :character   1st Qu.:   7.00   1st Qu.:46.21  
 Mode  :character   Median :  12.00   Median :46.21  
                    Mean   :  23.94   Mean   :46.21  
                    3rd Qu.:  27.00   3rd Qu.:46.21  
                    Max.   :2863.00   Max.   :46.21  

   PersonId             counts             sd       
 Length:55349       Min.   :  1.00   Min.   :15.01  
 Class :character   1st Qu.:  2.00   1st Qu.:15.01  
 Mode  :character   Median :  4.00   Median :15.01  
                    Mean   : 10.35   Mean   :15.01  
                    3rd Qu.: 12.00   3rd Qu.:15.01  
                    Max.   :263.00   Max.   :15.01  

In [ ]:
#get cleaned body weight vital data in ambulatory and inpatient from Observation
param1 = list( ClassConcept=1065220) # inpatient
param2 = list( ClassConcept=1065216) #outpatient
sql1 <- "select 
v.*, e.ClassConceptId
from Observation v join Encounter e on (v.EncounterId = e.Id)
where e.ClassConceptId = :ClassConcept and v.CodeConceptId in (800318,773683)
and v.NormalizedValueNumeric is not NULL
and NormalizedValueUOMConceptId not in (0,1067557,1067561)
and e.StatusConceptId not in (2983199,2506590,2983200,2506591,2506595)
and v.EffectiveDateTime is not NULL;
"
sql2 <- "select 
v.*, e.ClassConceptId
from Vital v join Encounter e on (v.EncounterId = e.Id)
where e.ClassConceptId = :ClassConcept and v.CodeConceptId in (800318,773683)
and v.NormalizedValueNumeric is not NULL
and NormalizedValueUOMConceptId not in (0,1067557,1067561)
and e.StatusConceptId not in (2983199,2506590,2983200,2506591,2506595)
and v.EffectiveDateTime is not NULL;
"
inpatient_bw_bh_vital <- load_sql_table(con,snapshot,query = sql1,params = param1) %>% collect()
amb__bw_bh_vital <- load_sql_table(con,snapshot, query = sql2,params = param2) %>% collect()
head(inpatient_bw_bh_vital)
head(amb__bw_bh_vital)